<a href="https://colab.research.google.com/github/Yeleao/NovelAI/blob/main/NovelAILeaks_API_Backend_(4chan_Ver_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

基于 4chan 魔改版 NovelAILeaks (naifu) 制作。[来源](https://boards.4channel.org/g/thread/89095460#p89097704)

使用官方前端 + 优化版后端，可突破75限制，支持所有模型。

Credit: https://t.me/StableDiffusion_CN https://t.me/exlolicon

Thanks: Anonymous, 炼铜术士, 神楽坂早苗️, Jonathan, 咕 咕, 猫又逆变器, Gaein nidb



In [ ]:
#@title ### 0. 检查 GPU 工作状态

!nvidia-smi

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title ### 1. 下载 Novel AI API 后端、模型 
#@markdown 如果下载速度太慢可尝试 restart

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar 
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

In [ ]:
#@title ### 2. 安装依赖
#@markdown 耐心等待安装完成

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared

In [ ]:
#@title ### 3. 启动模型
#@markdown 访问输出的映射地址（以 `trycloudflare.com` / `bore.pub` 结尾）即可。
#@markdown - 请等待模型加载完成（出现`Application startup complete`字样）再访问
#@markdown - cloudflare 提供的服务偶尔会出现请求超时，可换用 bore 隧道

%cd /content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969 

In [8]:
#@title ### 4. （可选）使用 7G 的 `animefull-latest` 模型运行
#@markdown 默认使用的是 4G 大小的 animefull-final-pruned 模型。如果想使用 7G 的 animefull-latest 模型，运行这个

%cd /content/
!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 127.0.0.1 --port=7860 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969 

/content
tar: animefull-latest.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
/content/naifu
env: DTYPE=float16
env: CLIP_CONTEXTS=3
env: AMP=1
env: MODEL=stable-diffusion
env: DEV=True
env: MODEL_PATH=models/animefull-latest
env: ENABLE_EMA=1
env: VAE_PATH=models/animevae.pt
env: PENULTIMATE=1
env: PYTHONDONTWRITEBYTECODE=1
env: SAVE_FILES=1
2022-10-15T10:38:25Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2022-10-15T10:38:25Z INF Requesting new quick Tunnel on trycloudflare.com...
2022-10-15T10:38:25.272368Z  INFO bore_cli::client: connected to server remote_port=46849
2022-10-15T10:38:25.272428Z  INFO bore_cli::clien

KeyboardInterrupt: ignored